# Enhanced Feature Engineering for Fuel Blend Property Prediction

This notebook implements advanced feature engineering techniques for predicting fuel blend properties. It includes:

- **Breakthrough Feature Engineering**: Non-linear transformations, advanced aggregations
- **Shell-specific Features**: RON-like blending, viscosity modeling, density corrections
- **Statistical Features**: Comprehensive descriptive statistics, skewness, kurtosis
- **Cross-property Interactions**: Component interactions and ratios
- **Advanced Scaling**: Multiple scaling strategies for different model types
- **Robust Ensemble**: Multiple algorithms with performance-based weighting

The goal is to achieve 90+ score through sophisticated feature engineering and ensemble modeling.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import Ridge, ElasticNet, HuberRegressor, BayesianRidge, TheilSenRegressor, RANSACRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
from scipy.stats import skew, kurtosis
from sklearn.metrics import mean_absolute_percentage_error

print("All libraries imported successfully!")
print("Ready for advanced feature engineering and ensemble modeling.")

In [ ]:
# Install missing packages if needed
try:
    import lightgbm
    print("✅ LightGBM available")
except ImportError:
    print("📦 Installing LightGBM...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "lightgbm"])

try:
    import xgboost
    print("✅ XGBoost available")
except ImportError:
    print("📦 Installing XGBoost...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])

try:
    import catboost
    print("✅ CatBoost available")
except ImportError:
    print("📦 Installing CatBoost...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "catboost"])

try:
    import pytorch_tabnet
    print("✅ TabNet available")
except ImportError:
    print("📦 TabNet not available - will skip this model")
    
print("✅ Package check complete!")

In [ ]:
# Core imports (always available)
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, ElasticNet, HuberRegressor
from scipy.stats import skew, kurtosis
from sklearn.metrics import mean_absolute_percentage_error

# Optional imports with graceful handling
available_models = {
    'lightgbm': False,
    'xgboost': False, 
    'catboost': False,
    'tabnet': False
}

try:
    from lightgbm import LGBMRegressor, early_stopping, log_evaluation
    available_models['lightgbm'] = True
    print("✅ LightGBM imported successfully")
except ImportError:
    print("⚠️ LightGBM not available - using RandomForest as primary model")

try:
    from xgboost import XGBRegressor
    available_models['xgboost'] = True
    print("✅ XGBoost imported successfully")
except ImportError:
    print("⚠️ XGBoost not available")

try:
    from catboost import CatBoostRegressor
    available_models['catboost'] = True
    print("✅ CatBoost imported successfully")
except ImportError:
    print("⚠️ CatBoost not available")

try:
    from pytorch_tabnet.tab_model import TabNetRegressor
    available_models['tabnet'] = True
    print("✅ TabNet imported successfully")
except ImportError:
    print("⚠️ TabNet not available")

print(f"\n📊 Available models: {sum(available_models.values())}/4 optional models loaded")
print("🚀 Core sklearn models always available - pipeline will work!")

In [ ]:
def create_breakthrough_features(df, pca_model=None, scaler=None, fit_transformers=True):
    """
    Optimized advanced feature engineering function for fuel blend property prediction.
    
    This function creates a comprehensive set of features including:
    - Non-linear transformations (sqrt, log, square)
    - Advanced weighted aggregations (harmonic, geometric means)
    - Shell-specific blending rules (RON, viscosity, density, RVP)
    - Statistical features (min, max, std, skew, kurtosis, IQR)
    - Cross-property interactions
    - PCA components
    - Fraction-based features (entropy, Gini coefficient)
    """
    
    # Create a copy to avoid fragmenting the original DataFrame
    df = df.copy()
    
    # Base features
    features = [f'Component{i}_fraction' for i in range(1, 6)]
    features += [f'Component{i}_Property{j}' for i in range(1, 6) for j in range(1, 11)]
    
    print(f"Starting with {len(features)} base features")
    
    # Prepare all new features in dictionaries first, then concat all at once
    new_features = {}
    
    # Enhanced interaction features with non-linear transformations
    print("Creating enhanced interaction features...")
    for i in range(1, 6):
        for j in range(1, 11):
            base_interaction = df[f'Component{i}_fraction'] * df[f'Component{i}_Property{j}']
            new_features[f'frac{i}_prop{j}'] = base_interaction
            new_features[f'frac{i}_prop{j}_sqrt'] = df[f'Component{i}_fraction'] * np.sqrt(np.abs(df[f'Component{i}_Property{j}']))
            new_features[f'frac{i}_prop{j}_log'] = df[f'Component{i}_fraction'] * np.log(np.abs(df[f'Component{i}_Property{j}']) + 1)
            new_features[f'frac{i}_prop{j}_square'] = df[f'Component{i}_fraction'] * (df[f'Component{i}_Property{j}'] ** 2)
            features.extend([f'frac{i}_prop{j}', f'frac{i}_prop{j}_sqrt', f'frac{i}_prop{j}_log', f'frac{i}_prop{j}_square'])
    
    # Advanced weighted features with multiple aggregation methods
    print("Creating advanced weighted features...")
    for j in range(1, 11):
        prop_cols = [f'Component{i}_Property{j}' for i in range(1, 6)]
        frac_cols = [f'Component{i}_fraction' for i in range(1, 6)]
        
        # Multiple weighted aggregations
        weighted_mean = sum(df[f'Component{i}_fraction'] * df[f'Component{i}_Property{j}'] for i in range(1, 6))
        new_features[f'weighted_mean_prop{j}'] = weighted_mean
        
        weighted_var = sum(df[f'Component{i}_fraction'] * (df[f'Component{i}_Property{j}'] - weighted_mean) ** 2 for i in range(1, 6))
        new_features[f'weighted_var_prop{j}'] = weighted_var
        
        # Harmonic mean (important for fuel properties)
        safe_props = [np.maximum(df[f'Component{i}_Property{j}'], 1e-6) for i in range(1, 6)]
        harmonic_mean = sum(df[f'Component{i}_fraction'] / safe_props[i-1] for i in range(1, 6))
        new_features[f'harmonic_mean_prop{j}'] = 1 / harmonic_mean
        
        # Geometric mean (for multiplicative properties)
        log_geo_mean = sum(df[f'Component{i}_fraction'] * np.log(safe_props[i-1]) for i in range(1, 6))
        new_features[f'geometric_mean_prop{j}'] = np.exp(log_geo_mean)
        
        # Component dominance with ranking
        frac_array = np.array([df[f'Component{i}_fraction'] for i in range(1, 6)])
        dominant_idx = np.argmax(frac_array, axis=0)
        new_features[f'dominant_prop{j}'] = df.apply(lambda row:
            row[f'Component{dominant_idx[row.name] + 1}_Property{j}'], axis=1)
        
        # Blend balance and diversity
        frac_data = df[frac_cols]
        new_features[f'blend_balance_prop{j}'] = 1 - frac_data.std(axis=1)
        new_features[f'blend_diversity_prop{j}'] = frac_data.std(axis=1) / (frac_data.mean(axis=1) + 1e-8)
        
        features.extend([
            f'weighted_mean_prop{j}', f'weighted_var_prop{j}', f'harmonic_mean_prop{j}',
            f'geometric_mean_prop{j}', f'dominant_prop{j}', f'blend_balance_prop{j}', f'blend_diversity_prop{j}'
        ])
    
    # Advanced statistics
    print("Creating advanced statistical features...")
    for j in range(1, 11):
        prop_cols = [f'Component{i}_Property{j}' for i in range(1, 6)]
        prop_data = df[prop_cols]
        
        new_features[f'min_prop{j}'] = prop_data.min(axis=1)
        new_features[f'max_prop{j}'] = prop_data.max(axis=1)
        new_features[f'mean_prop{j}'] = prop_data.mean(axis=1)
        new_features[f'std_prop{j}'] = prop_data.std(axis=1)
        new_features[f'median_prop{j}'] = prop_data.median(axis=1)
        new_features[f'skew_prop{j}'] = prop_data.apply(lambda row: skew(row), axis=1)
        new_features[f'kurtosis_prop{j}'] = prop_data.apply(lambda row: kurtosis(row), axis=1)
        new_features[f'range_prop{j}'] = new_features[f'max_prop{j}'] - new_features[f'min_prop{j}']
        new_features[f'iqr_prop{j}'] = prop_data.quantile(0.75, axis=1) - prop_data.quantile(0.25, axis=1)
        
        features.extend([
            f'min_prop{j}', f'max_prop{j}', f'mean_prop{j}',
            f'std_prop{j}', f'median_prop{j}', f'skew_prop{j}', f'kurtosis_prop{j}',
            f'range_prop{j}', f'iqr_prop{j}'
        ])
    
    # Shell-specific advanced features
    print("Creating Shell-specific blending features...")
    for j in range(1, 11):
        fractions = [df[f'Component{i}_fraction'] for i in range(1, 6)]
        props = [df[f'Component{i}_Property{j}'] for i in range(1, 6)]
        safe_props = [np.maximum(p, 1e-6) for p in props]
        
        # RON-like blending (non-linear octane)
        ron_blend = sum(f * (r ** 1.5) for f, r in zip(fractions, safe_props)) ** (1 / 1.5)
        new_features[f'ron_like_blend_prop{j}'] = ron_blend
        
        # Viscosity-like blending (logarithmic)
        log_visc_blend = sum(f * np.log(r) for f, r in zip(fractions, safe_props))
        new_features[f'log_visc_blend_prop{j}'] = log_visc_blend
        
        # Density-like blending (linear but with corrections)
        density_blend = sum(f * r for f, r in zip(fractions, safe_props))
        new_features[f'density_blend_prop{j}'] = density_blend
        
        # Reid vapor pressure-like (exponential)
        rvp_blend = sum(f * np.exp(r / 100) for f, r in zip(fractions, safe_props))
        new_features[f'rvp_blend_prop{j}'] = rvp_blend
        
        features.extend([
            f'ron_like_blend_prop{j}', f'log_visc_blend_prop{j}',
            f'density_blend_prop{j}', f'rvp_blend_prop{j}'
        ])
    
    # Cross-property interactions (most important combinations)
    print("Creating cross-property interactions...")
    for j1 in range(1, 6):
        for j2 in range(j1 + 1, 7):
            if j2 <= 10:  # Make sure we don't exceed property range
                interaction = new_features[f'weighted_mean_prop{j1}'] * new_features[f'weighted_mean_prop{j2}']
                ratio = new_features[f'weighted_mean_prop{j1}'] / (new_features[f'weighted_mean_prop{j2}'] + 1e-8)
                new_features[f'prop{j1}_prop{j2}_interaction'] = interaction
                new_features[f'prop{j1}_prop{j2}_ratio'] = ratio
                features.extend([f'prop{j1}_prop{j2}_interaction', f'prop{j1}_prop{j2}_ratio'])
    
    # Enhanced PCA with more components
    print("Creating PCA features...")
    prop_features = [f'Component{i}_Property{j}' for i in range(1, 6) for j in range(1, 11)]
    if fit_transformers:
        pca = PCA(n_components=12, random_state=42)
        pca_feats = pca.fit_transform(df[prop_features])
    else:
        pca = pca_model
        pca_feats = pca.transform(df[prop_features])
    
    for k in range(12):
        new_features[f'pca_prop_{k+1}'] = pca_feats[:, k]
        features.append(f'pca_prop_{k+1}')
    
    # Fraction-based advanced features
    print("Creating fraction-based features...")
    frac_cols = [f'Component{i}_fraction' for i in range(1, 6)]
    frac_data = df[frac_cols]
    
    new_features['frac_sum'] = frac_data.sum(axis=1)
    new_features['frac_std'] = frac_data.std(axis=1)
    new_features['frac_skew'] = frac_data.apply(lambda row: skew(row), axis=1)
    new_features['frac_kurtosis'] = frac_data.apply(lambda row: kurtosis(row), axis=1)
    new_features['frac_entropy'] = -sum(df[f'Component{i}_fraction'] * np.log(df[f'Component{i}_fraction'] + 1e-8) for i in range(1, 6))
    new_features['frac_gini'] = 1 - sum(df[f'Component{i}_fraction'] ** 2 for i in range(1, 6))
    
    features.extend(['frac_sum', 'frac_std', 'frac_skew', 'frac_kurtosis', 'frac_entropy', 'frac_gini'])
    
    # Convert new features to DataFrame and concatenate all at once
    print("Combining all features...")
    new_features_df = pd.DataFrame(new_features, index=df.index)
    df_enhanced = pd.concat([df, new_features_df], axis=1)
    
    print(f"Feature engineering complete! Created {len(features)} total features.")
    return df_enhanced, features, pca

print("Optimized feature engineering function defined successfully!")

In [ ]:
# Load Data
print("Loading data...")
try:
    # Try to load from current directory first
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    print("✅ Data loaded from current directory")
except FileNotFoundError:
    try:
        # Try to load from dataset directory
        train = pd.read_csv('../../dataset/train.csv')
        test = pd.read_csv('../../dataset/test.csv')
        print("✅ Data loaded from dataset directory")
    except FileNotFoundError:
        print("❌ Error: train.csv and test.csv not found. Please ensure the data files are available.")
        raise

print(f"Training data shape: {train.shape}")
print(f"Test data shape: {test.shape}")
print(f"Training columns: {list(train.columns)}")

# Apply breakthrough feature engineering
print("\n" + "="*60)
print("STARTING BREAKTHROUGH FEATURE ENGINEERING")
print("="*60)

train_enhanced, feat_cols, pca_model = create_breakthrough_features(train, fit_transformers=True)
test_enhanced, _, _ = create_breakthrough_features(test, pca_model=pca_model, fit_transformers=False)

print(f"\n✅ Feature engineering complete!")
print(f"Original features: 55 (5 fractions + 50 properties)")
print(f"Enhanced features: {len(feat_cols)}")
print(f"Feature increase: {len(feat_cols) - 55}x more features")

In [ ]:
# Prepare Data for Training
print("\n" + "="*60)
print("DATA PREPARATION AND FEATURE SCALING")
print("="*60)

# Define target variables
TARGETS = [f'BlendProperty{i}' for i in range(1, 11)]
print(f"Target variables: {TARGETS}")

# Prepare training and test data
X_train = train_enhanced[feat_cols]
y_train = train_enhanced[TARGETS]
X_test = test_enhanced[feat_cols]

print(f"Training features shape: {X_train.shape}")
print(f"Training targets shape: {y_train.shape}")
print(f"Test features shape: {X_test.shape}")

# Handle NaN values
print("\nHandling NaN values...")
nan_before_train = X_train.isnull().sum().sum()
nan_before_test = X_test.isnull().sum().sum()

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print(f"NaN values in training data: {nan_before_train} -> 0")
print(f"NaN values in test data: {nan_before_test} -> 0")

# Feature scaling for different model types
print("\nCreating multiple scaling strategies...")

# Robust scaling (less sensitive to outliers)
scaler_robust = RobustScaler()
X_train_robust = scaler_robust.fit_transform(X_train)
X_test_robust = scaler_robust.transform(X_test)
print("✅ Robust scaling complete")

# Standard scaling (for linear models)
scaler_standard = StandardScaler()
X_train_standard = scaler_standard.fit_transform(X_train)
X_test_standard = scaler_standard.transform(X_test)
print("✅ Standard scaling complete")

# Feature selection for computational efficiency
print("\nPerforming feature selection...")
selector = SelectFromModel(
    LGBMRegressor(n_estimators=200, random_state=42, verbose=-1),
    prefit=False,
    threshold='median'
)

X_train_selected = selector.fit_transform(X_train, y_train.iloc[:, 0])
X_test_selected = selector.transform(X_test)

selected_features = [feat_cols[i] for i in range(len(feat_cols)) if selector.get_support()[i]]
print(f"✅ Feature selection complete:")
print(f"  Original features: {len(feat_cols)}")
print(f"  Selected features: {len(selected_features)}")
print(f"  Reduction: {(1 - len(selected_features)/len(feat_cols))*100:.1f}%")

In [ ]:
# Advanced Ensemble Training with Cross-Validation
print("\n" + "="*60)
print("BREAKTHROUGH ENSEMBLE TRAINING")
print("="*60)

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
final_preds = np.zeros((X_test.shape[0], len(TARGETS)))

# Check if LightGBM is available
try:
    from lightgbm import LGBMRegressor, early_stopping, log_evaluation
    lgb_available = True
    print("✅ LightGBM available")
except ImportError:
    lgb_available = False
    print("⚠️ LightGBM not available - using RandomForest as primary model")

print(f"Cross-validation: {kf.get_n_splits()}-fold")
print(f"Features: {len(feat_cols)} (selected: {len(selected_features)})")
print(f"Models: 7 core models with advanced ensemble weighting")

for i, target in enumerate(TARGETS):
    print(f"\n{'='*40}")
    print(f"Training for {target} ({i+1}/{len(TARGETS)})")
    print(f"{'='*40}")
    
    # Initialize out-of-fold predictions for each model
    primary_oof = np.zeros(X_train.shape[0])  # LightGBM or RandomForest
    rf_oof = np.zeros(X_train.shape[0])
    et_oof = np.zeros(X_train.shape[0])
    gb_oof = np.zeros(X_train.shape[0])
    ridge_oof = np.zeros(X_train.shape[0])
    elastic_oof = np.zeros(X_train.shape[0])
    huber_oof = np.zeros(X_train.shape[0])
    
    # Initialize test predictions for each model
    primary_test_preds = np.zeros(X_test.shape[0])
    rf_test_preds = np.zeros(X_test.shape[0])
    et_test_preds = np.zeros(X_test.shape[0])
    gb_test_preds = np.zeros(X_test.shape[0])
    ridge_test_preds = np.zeros(X_test.shape[0])
    elastic_test_preds = np.zeros(X_test.shape[0])
    huber_test_preds = np.zeros(X_test.shape[0])
    
    # Cross-validation training
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"  Fold {fold + 1}/{kf.get_n_splits()}", end=" - ")
        
        # Model 1: Primary model (LightGBM if available, otherwise RandomForest)
        if lgb_available:
            model_primary = LGBMRegressor(
                n_estimators=5000,  # Reduced from 12000
                learning_rate=0.01,  # Increased from 0.0015 for faster convergence
                random_state=fold,
                num_leaves=31, 
                subsample=0.85, 
                colsample_bytree=0.85,
                reg_alpha=0.01, 
                reg_lambda=0.01, 
                min_child_samples=20,
                objective='regression_l1',
                verbose=-1  # Silent training
            )
            
            model_primary.fit(
                X_train.iloc[tr_idx], y_train[target].iloc[tr_idx],
                eval_set=[(X_train.iloc[val_idx], y_train[target].iloc[val_idx])],
                callbacks=[early_stopping(stopping_rounds=100), log_evaluation(0)]
            )
            primary_name = "LightGBM"
        else:
            model_primary = RandomForestRegressor(
                n_estimators=1000, max_depth=20, min_samples_split=5,
                min_samples_leaf=2, random_state=fold, n_jobs=-1
            )
            model_primary.fit(X_train.iloc[tr_idx], y_train[target].iloc[tr_idx])
            primary_name = "RandomForest"
        
        primary_oof[val_idx] = model_primary.predict(X_train.iloc[val_idx])
        primary_test_preds += model_primary.predict(X_test) / kf.get_n_splits()
        
        # Model 2: Random Forest (if not primary)
        if lgb_available:
            model_rf = RandomForestRegressor(
                n_estimators=800, max_depth=20, min_samples_split=5,
                min_samples_leaf=2, random_state=fold, n_jobs=-1
            )
            model_rf.fit(X_train.iloc[tr_idx], y_train[target].iloc[tr_idx])
            rf_oof[val_idx] = model_rf.predict(X_train.iloc[val_idx])
            rf_test_preds += model_rf.predict(X_test) / kf.get_n_splits()
        else:
            # Use Extra Trees as second model if LightGBM not available
            rf_oof[val_idx] = primary_oof[val_idx]  # Duplicate primary for ensemble
            rf_test_preds += primary_test_preds / kf.get_n_splits()
        
        # Model 3: Extra Trees
        model_et = ExtraTreesRegressor(
            n_estimators=600, max_depth=18, min_samples_split=3,
            min_samples_leaf=1, random_state=fold, n_jobs=-1
        )
        model_et.fit(X_train.iloc[tr_idx], y_train[target].iloc[tr_idx])
        et_oof[val_idx] = model_et.predict(X_train.iloc[val_idx])
        et_test_preds += model_et.predict(X_test) / kf.get_n_splits()
        
        # Model 4: Gradient Boosting
        model_gb = GradientBoostingRegressor(
            n_estimators=300,  # Reduced for faster training
            learning_rate=0.05,  # Increased for faster convergence
            max_depth=6,
            min_samples_split=5, 
            min_samples_leaf=2, 
            random_state=fold
        )
        model_gb.fit(X_train.iloc[tr_idx], y_train[target].iloc[tr_idx])
        gb_oof[val_idx] = model_gb.predict(X_train.iloc[val_idx])
        gb_test_preds += model_gb.predict(X_test) / kf.get_n_splits()
        
        # Model 5: Ridge (with robust scaling)
        model_ridge = Ridge(alpha=0.1, random_state=fold)  # Increased alpha
        model_ridge.fit(X_train_robust[tr_idx], y_train[target].iloc[tr_idx])
        ridge_oof[val_idx] = model_ridge.predict(X_train_robust[val_idx])
        ridge_test_preds += model_ridge.predict(X_test_robust) / kf.get_n_splits()
        
        # Model 6: Elastic Net (with standard scaling)
        model_elastic = ElasticNet(
            alpha=0.01, 
            l1_ratio=0.3, 
            random_state=fold, 
            max_iter=5000  # Increased iterations
        )
        model_elastic.fit(X_train_standard[tr_idx], y_train[target].iloc[tr_idx])
        elastic_oof[val_idx] = model_elastic.predict(X_train_standard[val_idx])
        elastic_test_preds += model_elastic.predict(X_test_standard) / kf.get_n_splits()
        
        # Model 7: Huber (robust to outliers) - Fixed convergence issue
        model_huber = HuberRegressor(
            alpha=0.1,  # Increased from 0.01 
            epsilon=1.35,
            max_iter=500,  # Increased from default
            tol=1e-3  # More lenient tolerance
        )
        model_huber.fit(X_train_robust[tr_idx], y_train[target].iloc[tr_idx])
        huber_oof[val_idx] = model_huber.predict(X_train_robust[val_idx])
        huber_test_preds += model_huber.predict(X_test_robust) / kf.get_n_splits()
        
        print("✅")
    
    # Calculate individual model MAPE scores
    primary_mape = mean_absolute_percentage_error(y_train[target], primary_oof)
    rf_mape = mean_absolute_percentage_error(y_train[target], rf_oof)
    et_mape = mean_absolute_percentage_error(y_train[target], et_oof)
    gb_mape = mean_absolute_percentage_error(y_train[target], gb_oof)
    ridge_mape = mean_absolute_percentage_error(y_train[target], ridge_oof)
    elastic_mape = mean_absolute_percentage_error(y_train[target], elastic_oof)
    huber_mape = mean_absolute_percentage_error(y_train[target], huber_oof)
    
    # Advanced ensemble: Exponential weighting based on validation performance
    mape_scores = [primary_mape, rf_mape, et_mape, gb_mape, ridge_mape, elastic_mape, huber_mape]
    weights = [np.exp(-score * 10) for score in mape_scores]  # Exponential weighting
    total_weight = sum(weights)
    weights = [w / total_weight for w in weights]
    
    # Final ensemble predictions
    final_preds[:, i] = (
        weights[0] * primary_test_preds +
        weights[1] * rf_test_preds +
        weights[2] * et_test_preds +
        weights[3] * gb_test_preds +
        weights[4] * ridge_test_preds +
        weights[5] * elastic_test_preds +
        weights[6] * huber_test_preds
    )
    
    # Calculate ensemble validation score
    ensemble_oof = (
        weights[0] * primary_oof +
        weights[1] * rf_oof +
        weights[2] * et_oof +
        weights[3] * gb_oof +
        weights[4] * ridge_oof +
        weights[5] * elastic_oof +
        weights[6] * huber_oof
    )
    
    ensemble_mape = mean_absolute_percentage_error(y_train[target], ensemble_oof)
    
    # Display results
    print(f"\n  📊 {target} Results:")
    print(f"     {primary_name}:     {primary_mape:.4f} (weight: {weights[0]:.3f})")
    print(f"     Random Forest: {rf_mape:.4f} (weight: {weights[1]:.3f})")
    print(f"     Extra Trees:   {et_mape:.4f} (weight: {weights[2]:.3f})")
    print(f"     Gradient Boost: {gb_mape:.4f} (weight: {weights[3]:.3f})")
    print(f"     Ridge:        {ridge_mape:.4f} (weight: {weights[4]:.3f})")
    print(f"     Elastic Net:  {elastic_mape:.4f} (weight: {weights[5]:.3f})")
    print(f"     Huber:        {huber_mape:.4f} (weight: {weights[6]:.3f})")
    print(f"     🎯 ENSEMBLE:   {ensemble_mape:.4f}")

print(f"\n🎉 Training complete! All {len(TARGETS)} targets processed.")

In [ ]:
# Create Submission File
print("\n" + "="*60)
print("CREATING SUBMISSION FILE")
print("="*60)

# Create submission dataframe
submission = pd.DataFrame(final_preds, columns=TARGETS)

# Add ID column (assuming test data has ID column, otherwise create sequential IDs)
if 'ID' in test_enhanced.columns:
    submission.insert(0, 'ID', test_enhanced['ID'])
    print("✅ Using existing ID column from test data")
else:
    submission.insert(0, 'ID', np.arange(1, len(test_enhanced) + 1))
    print("✅ Created sequential ID column")

# Save submission
submission_filename = 'submission_enhanced_breakthrough.csv'
submission.to_csv(submission_filename, index=False)

print(f"✅ Submission saved as: {submission_filename}")
print(f"📊 Submission shape: {submission.shape}")

# Display sample predictions
print("\\n📋 Sample predictions:")
print(submission.head())

print("\n" + "="*60)
print("🎯 BREAKTHROUGH ENSEMBLE SUMMARY")
print("="*60)
print(f"🔬 Features: {len(feat_cols)} (selected: {len(selected_features)})")
print(f"🔄 Cross-validation: {kf.get_n_splits()}-fold")
print(f"🤖 Models: LightGBM, Random Forest, Extra Trees, Gradient Boosting, Ridge, Elastic Net, Huber")
print(f"⚖️  Ensemble: Exponential weighting based on validation performance")
print(f"📏 Scaling: Robust and Standard scaling for different model types")
print(f"🎯 Target: 90+ score with breakthrough features and advanced ensemble")
print(f"📄 Output: {submission_filename}")
print("="*60)

print("\\n🚀 Ready for submission! The enhanced pipeline includes:")
print("   • Advanced non-linear transformations")
print("   • Shell-specific fuel blending rules")
print("   • Comprehensive statistical features")
print("   • Cross-property interactions")
print("   • Multiple scaling strategies")
print("   • Performance-weighted ensemble")

In [ ]:
# Feature Importance Analysis (Optional)
print("\n" + "="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Train a single LightGBM model on all data to get feature importance
print("Training feature importance model...")
importance_model = LGBMRegressor(
    n_estimators=1000, 
    random_state=42, 
    verbose=-1
)

# Use the first target for importance analysis
importance_model.fit(X_train, y_train.iloc[:, 0])

# Get feature importance
feature_importance = importance_model.feature_importances_
feature_names = feat_cols

# Create importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\\n🔝 Top 20 Most Important Features:")
print(importance_df.head(20).to_string(index=False))

# Analyze feature types
feature_types = {
    'base': 0, 'interaction': 0, 'statistical': 0, 
    'shell_specific': 0, 'pca': 0, 'fraction': 0
}

for feature in importance_df['feature'].head(50):  # Top 50 features
    if 'frac' in feature and 'prop' in feature:
        feature_types['interaction'] += 1
    elif any(stat in feature for stat in ['min_', 'max_', 'mean_', 'std_', 'skew_', 'kurtosis_']):
        feature_types['statistical'] += 1
    elif any(blend in feature for blend in ['ron_like', 'log_visc', 'density_blend', 'rvp_blend']):
        feature_types['shell_specific'] += 1
    elif 'pca' in feature:
        feature_types['pca'] += 1
    elif feature.startswith('frac_'):
        feature_types['fraction'] += 1
    else:
        feature_types['base'] += 1

print("\\n📊 Feature Type Distribution (Top 50):")
for ftype, count in feature_types.items():
    print(f"   {ftype.replace('_', ' ').title()}: {count}")

print("\\n💡 Insights:")
if feature_types['interaction'] > 10:
    print("   • Interaction features are highly valuable")
if feature_types['shell_specific'] > 5:
    print("   • Shell-specific blending rules are important")
if feature_types['statistical'] > 8:
    print("   • Statistical aggregations provide good signal")
if feature_types['pca'] > 3:
    print("   • PCA components capture meaningful patterns")